In [11]:
import time
from design import *
import importlib
import shutil
from utils import *
from openai import OpenAI
from prompts import *
import json
import numpy as np
from gymnasium.envs.robodesign.GPTSwimmer import GPTSwimmerEnv
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import os
import matplotlib.pyplot as plt


In [12]:
def load_logs_full(log_dir):
    # 找到 .tfevents 文件
    event_file = [f for f in os.listdir(log_dir) if f.startswith("events.out")][0]
    event_path = os.path.join(log_dir, event_file)

    # 加载日志
    event_acc = EventAccumulator(event_path)
    event_acc.Reload()

    # 获取所有 scalar tags
    all_tags = event_acc.Tags()["scalars"]

    # 筛选 reward 分量
    reward_tags = [tag for tag in all_tags if tag.startswith("reward/")]

    # 加载所有 reward 分量数据（完整）
    data_full = {}
    for tag in reward_tags:
        events = event_acc.Scalars(tag)
        values = [e.value for e in events]
        data_full[tag] = values

    # 加载 episode length（完整）
    ep_len_tag = "rollout/ep_len_mean"
    if ep_len_tag in all_tags:
        events = event_acc.Scalars(ep_len_tag)
        values = [e.value for e in events]
        data_full[ep_len_tag] = values

    return data_full


In [ ]:
import prompts
class DGA:
    def __init__(self):
        api_key = "<api_key>"
        self.client = OpenAI(api_key=api_key)
        self.model = "gpt-4o-mini"

    def extract_code(self, text):
        match = re.search(r'```python\n(.*?)\n```', text, re.DOTALL)
        return match.group(1).strip() if match else None

    def indent_code(self, code):
        return "\n".join(line if line.strip() else line for line in code.split("\n"))

    def generate_rewardfunc_eureka(self, rewardfunc_nums, best_message, iteration, folder_name):
        messages = [
            {"role": "system", "content": "You are a reinforcement learning reward function designer"},
            {"role": "user", "content": rewardfunc_prompts + best_message + zeroshot_rewardfunc_format}
        ]

        responses = self.client.chat.completions.create(
            model=self.model, messages=messages, n=rewardfunc_nums
        )
        
        files = []
        for i, choice in enumerate(responses.choices):
            reward_code = self.extract_code(choice.message.content)
            if reward_code:
                full_code = "import numpy as np \n" + self.indent_code(reward_code) + "\n"
                file_name =  f"GPTrewardfunc_{i}_{iteration}.py"
                file_path = os.path.join(folder_name, "env", file_name)

                with open(file_path, "w") as fp:
                    fp.write(full_code)
                files.append(file_path)
                print(f"Saved: {file_path}")

        return files


In [14]:

folder_name = "results/eureka"
log_file = os.path.join(folder_name, "parameters.log")
logging.basicConfig(filename=log_file, level=logging.INFO, format="%(asctime)s - %(message)s")

# folder_name = setup_logging(div_flag=True)

best_fitness = float('-inf')  
best_morphology = None  
best_rewardfunc = None  
best_reward = None
best_material = None
best_efficiency = None

iterations = 5
morphology_nums = 1
rewardfunc_nums = 16

fitness_matrix = np.array([[None for _ in range(morphology_nums)] for _ in range(rewardfunc_nums)])
efficiency_matrix = np.array([[None for _ in range(morphology_nums)] for _ in range(rewardfunc_nums)])
fitness_list = []


In [15]:
morphology_list = [f'results/eureka/assets/GPTSwimmer_{i}.xml' for i in range(0,1) ]
parameter_list = [[1, 1, 1, 0.1, 0.1, 0.1]]
material_list = [compute_swimmer_volume(parameter) for parameter in parameter_list]


In [16]:
eureka_rewardfunc_prompts = """We trained a RL policy using the provided function code and tracked
the values of the individual components in the reward function as well as global policy matrics such as fitness function and episode lengths after
10000 epochs and the maximum, mean, and minimum values encountered:
{reward_reflection}

Please carefully analyze the policy feedback and provide a new, improved reward function that can better
solve the task. Some helpful tips for analyzing the policy feedback:

(1) If the fitness function rates are always zero or negative, then you must rewrite the entire reward function
(2) If the values for a certain reward component are near identiacal throughout, then this means RL is not able to optimize this component as it is written. You may 
consider 
    (a) Changing its scale or the value of its temperature parameter
    (b) Re-writing the reward component
    (c) Discarding the reward component
(3) If some reward components' magnitude is significanly larger, then you must rescale its value to a proper range
Please analyze each existing reward component in the suggested manner above first, and then write the reward function code
"""

In [17]:
designer = DGA()
best_message = ''

for iter in range(1, iterations+1):
    reward_reflection = ''
    rewardfunc_list = designer.generate_rewardfunc_eureka(rewardfunc_nums, best_message, iter, folder_name)
    logging.info(f"___________________coarse optimization iter{iter}_____________________")
    for i, rewardfunc in enumerate(rewardfunc_list):
        for j, morphology in enumerate(morphology_list):

            print(i, rewardfunc)
            print(j, morphology)

            shutil.copy(morphology, "GPTSwimmer.xml")
            shutil.copy(rewardfunc, "GPTrewardfunc.py")         

            import GPTrewardfunc
            importlib.reload(GPTrewardfunc)  # 重新加载模块
            from GPTrewardfunc import _get_rew
            GPTSwimmerEnv._get_rew = _get_rew

            env_name = "GPTSwimmerEnv"
            model_path = Train(j,  i, folder_name, total_timesteps=5e5, callback=True, iter=iter)
            fitness, reward = Eva(model_path,run_steps=100)

            material = material_list[j]
            efficiency = fitness/material
            fitness_matrix[i][j] = fitness
            efficiency_matrix[i][j] = efficiency

            logging.info(f"iteration:{iter}, morphology: {j}, rewardfunc: {i}, material cost: {material} reward: {reward} fitness: {fitness} efficiency: {efficiency}")

    best_efficiency = np.max(efficiency_matrix[:, 0])
    best_rewardfunc_index = np.argmax(efficiency_matrix[:, 0])
    logs_full = load_logs_full(f"results/eureka/sac_morphology0_rewardfunc{best_rewardfunc_index}_{iter}/SAC_1")  
    epoch_freq = 100000

    for tag, full_values in logs_full.items():
        sampled_values = full_values[::epoch_freq]
        max_val = max(sampled_values)
        mean_val = sum(sampled_values) / len(sampled_values)
        min_val = min(sampled_values)
        formatted_values = [f"{v:.2f}" for v in sampled_values]
        reward_reflection =f"{tag}: {formatted_values}" + f"Max: {max_val:.2f}, Mean: {mean_val:.2f}, Min: {min_val:.2f}\n"
        
    with open(best_rewardfunc, 'r') as f:
        reward_content = f.read()
    best_message = f"best reward:{reward_content} \n" + f"best fintess:{best_efficiency}" + eureka_rewardfunc_prompts.format(reward_reflection=reward_reflection)


InternalServerError: Error code: 524 - {'error': {'message': 'bad response status code 524 (request id: 20250409054617273693001Xdwmp8fK)', 'type': 'upstream_error', 'param': '524', 'code': 'bad_response_status_code'}}

In [ ]:
efficiency_matrix

In [10]:
best_efficiency

18.651636198983123

In [38]:

logs_full = load_logs_full(f"results/eureka/sac_morphology0_rewardfunc7/SAC_1")  
# logs_full = load_logs_full(f"results\eureka\")  
epoch_freq = 100000

for tag, full_values in logs_full.items():
    sampled_values = full_values[::epoch_freq]
    max_val = max(sampled_values)
    mean_val = sum(sampled_values) / len(sampled_values)
    min_val = min(sampled_values)
    formatted_values = [f"{v:.2f}" for v in sampled_values]
    reward_reflection +=f"{tag}: {formatted_values}" + f"Max: {max_val:.2f}, Mean: {mean_val:.2f}, Min: {min_val:.2f}\n"

NameError: name 'reward_reflection' is not defined

results/eureka
results/eureka/assets
results/eureka/assets/.ipynb_checkpoints
results/eureka/.ipynb_checkpoints
results/eureka/env
results/eureka/env/.ipynb_checkpoints
results/eureka/sac_morphology0_rewardfunc0
results/eureka/sac_morphology0_rewardfunc0/SAC_1
results/eureka/coarse
results/eureka/coarse/.ipynb_checkpoints
results/eureka/sac_morphology0_rewardfunc1
results/eureka/sac_morphology0_rewardfunc1/SAC_1
results/eureka/sac_morphology0_rewardfunc2
results/eureka/sac_morphology0_rewardfunc2/SAC_1
results/eureka/sac_morphology0_rewardfunc3
results/eureka/sac_morphology0_rewardfunc3/SAC_1
results/eureka/sac_morphology0_rewardfunc4
results/eureka/sac_morphology0_rewardfunc4/SAC_1
results/eureka/sac_morphology0_rewardfunc5
results/eureka/sac_morphology0_rewardfunc5/SAC_1
results/eureka/sac_morphology0_rewardfunc6
results/eureka/sac_morphology0_rewardfunc6/SAC_1
results/eureka/sac_morphology0_rewardfunc7
results/eureka/sac_morphology0_rewardfunc7/SAC_1
results/eureka/sac_morphology0_r